In [1]:
import logging

# Create a custom logger
logger = logging.getLogger(__name__)

# Set the level of this logger. 
# DEBUG, INFO, WARNING, ERROR, CRITICAL are the logging levels in increasing order of severity.
logger.setLevel(logging.DEBUG)

# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.DEBUG)

# Create formatters and add it to handlers
c_format = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
c_handler.setFormatter(c_format)

# Add handlers to the logger
logger.addHandler(c_handler)

logger.info('This is an info message')

__main__ - INFO - This is an info message


In [2]:
import subprocess
import dspy

class IterativeCodeRefinement(dspy.Module):
    def __init__(self):
        super().__init__()
        
        self.logger = logging.getLogger('my_logger')
        self.logger.setLevel(logging.DEBUG)
        
        handler = logging.StreamHandler()
        handler.setLevel(logging.DEBUG)
        

        formatter = logging.Formatter('%(message)s')
        handler.setFormatter(formatter)

        self.logger.addHandler(handler)
        
        self.logger.info("Testing Log")
        self.generate_pseudocode = dspy.ChainOfThought("task -> pseudocode")
        self.pseudocode_to_code = dspy.ChainOfThought("task, pseudocode -> code")
        self.generate_code_tests = dspy.ChainOfThought("task, code -> tests")
        self.check_code_correctness = dspy.ChainOfThought("code, tests, code_execution_output -> yes/no")
        self.refine_pseudocode = dspy.ChainOfThought("code output, test output, errors -> new pseudocode")
        self.refine_code_with_previous_context = dspy.ChainOfThought("task, new pseudocode, previous code+errors -> new code")
        self.refine_tests_with_previous_context = dspy.ChainOfThought("task, new code, previous tests+errors -> new tests")
    
    def execute_code(self, code):
        # Assume code and tests are combined into the code variable
        result = subprocess.run(["python", "-c", code], capture_output=True, text=True, shell=True)
        return result.stdout, result.stderr, result.returncode

    def forward(self, task):
        pseudocode = self.generate_pseudocode(task=task).pseudocode
        code = self.pseudocode_to_code(task=task, pseudocode=pseudocode).code
        tests = self.generate_code_tests(task=task, code=code).tests
    
        self.logger.info(f"pseudocode: {pseudocode}")
        self.logger.info(f"code: {code}")
        self.logger.info(f"tests: {tests}")
        
        # Combine code and tests for execution
        combined_code = code + "\n" + tests
        stdout, stderr, returncode = self.execute_code(combined_code)
        self.logger.info(f"stdout: {stdout}")

        # Initial check for correctness
        is_correct = self.check_code_correctness(code=code, tests=tests, code_execution_output=stdout + stderr).answer
        self.logger.info(f"is_correct: {is_correct}")

        # Loop for refinement based on the condition
        while is_correct.lower() != "yes":
            new_pseudocode = self.refine_pseudocode(code_output=stdout, test_output=stderr, errors=str(returncode)).pseudocode
            new_code = self.refine_code_with_previous_context(task=task, new_pseudocode=new_pseudocode, previous_code_errors=stderr).code
            new_tests = self.refine_tests_with_previous_context(task=task, new_code=new_code, previous_tests_errors=stderr).tests
            
            self.logger.info(f"new_pseudocode: {new_pseudocode}")
            self.logger.info(f"new_code: {new_code}")
            self.logger.info(f"new_tests: {new_tests}") 
            print(f"new psuedocode: {new_pseudocode}")
            combined_code = new_code + "\n" + new_tests
            stdout, stderr, returncode = self.execute_code(combined_code)
            is_correct = self.check_code_correctness(code=new_code, tests=new_tests, code_execution_output=stdout + stderr).answer

            self.logger.info(f"stdout: {stdout}")
            self.logger.info(f"is_correct: {is_correct}")
            
        return {"final_code": new_code, "final_tests": new_tests, "output": stdout, "errors": stderr}

In [3]:
llm = dspy.OllamaLocal("open-hermes-2-4_0", max_tokens=3000, model_type="chat")
dspy.settings.configure(lm=llm)

In [4]:
IterativeCodeRefinement()(task="Write a python function that adds 2 numbers and returns the result.")

Testing Log
pseudocode: ```python
def add_numbers(number1, number2):
    # Step 1: Perform addition of two numbers
    sum = number1 + number2
    
    # Step 2: Return the result
    return sum
```
code: def add_numbers(number1, number2):
    # Step 1: Perform addition of two numbers
    sum = number1 + number2
    # Step 2: Return the result
    return sum
tests: 1. Test case 1: add_numbers(2, 3) should return 5
2. Test case 2: add_numbers(-2, 4) should return 2
3. Test case 3: add_numbers(0, 0) should return 0
4. Test case 4: add_numbers(10, -5) should return 5
5. Test case 5: add_numbers('2', '3') should raise a TypeError
6. Test case 6: add_numbers([2], [3]) should raise a TypeError


KeyboardInterrupt: 